# Deep Learning on Your Own Computer
By. Afif A. Iskandar

## Membuat klasifier citra untuk kasus pembalap MotoGP

## Melatih CNN

In [1]:
# import library yang dibutuhkan
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from neural_net.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import os
import pickle
import matplotlib

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
def list_images(basePath, contains=None):
    # return the set of files that are valid
    return list_files(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=contains)

def list_files(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                yield imagePath

In [4]:
# Definisikan path dataset dan tempat menyimpan model
dataset = 'dataset/'
model_path = 'gprider.pkl'
labelbin = 'lb.pickle'


In [5]:
#  hyperparameter
EPOCHS = 500
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

In [6]:
# inisialisasi list data dan label
data = []
labels = []

# load gambar dan acak untuk train + test
imagePaths = sorted(list(list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)

In [7]:
# loop pada dataset
for imagePath in imagePaths:
    image = load_img(imagePath, target_size=(IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # ekstrak label
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [8]:
# Normalisasi data
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matriks: {:2f}MB".format(
    data.nbytes/ (1024 * 1000.0)))


[INFO] data matriks: 102.600000MB


In [9]:
# binarisasi label
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [10]:
# train-test-split
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.2, random_state=42)

In [11]:
# konstruksi ImageGenerator
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [ ]:
# inisialisasi model
print("[INFO] Mengcompile Jaringan")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], 
    height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], 
    classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

[INFO] Mengcompile Jaringan


In [ ]:
# latih jaringan
print("[INFO] Melatih Jaringan")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] Melatih Jaringan
Epoch 1/500
11/11 [==============================] - 144s 13s/step - loss: 2.7226 - acc: 0.2927 - val_loss: 3.8367 - val_acc: 0.3579
Epoch 2/500
11/11 [==============================] - 138s 13s/step - loss: 2.0584 - acc: 0.3823 - val_loss: 4.5414 - val_acc: 0.2737
Epoch 3/500
11/11 [==============================] - 152s 14s/step - loss: 1.6555 - acc: 0.4282 - val_loss: 4.5795 - val_acc: 0.3053
Epoch 4/500
11/11 [==============================] - 164s 15s/step - loss: 1.5963 - acc: 0.4638 - val_loss: 1.8588 - val_acc: 0.4000
Epoch 5/500
11/11 [==============================] - 150s 14s/step - loss: 1.2970 - acc: 0.5256 - val_loss: 2.6989 - val_acc: 0.4737
Epoch 6/500
11/11 [==============================] - 146s 13s/step - loss: 1.2103 - acc: 0.5641 - val_loss: 2.6955 - val_acc: 0.4737
Epoch 7/500
11/11 [==============================] - 142s 13s/step - loss: 1.0677 - acc: 0.6209 - val_loss: 1.7349 - val_acc: 0.5053
Epoch 8/500
11/11 [==========================

In [ ]:
# menyimpan model
print("[INFO] Menyimpan Model Kedalam Disk")
model.save(model_path)

In [ ]:
# menyimpan label binary
print("[INFO] Menyimpan Label Binary")
f = open(labelbin, 'wb')
f.write(pickle.dumps(lb))
f.close()

In [ ]:
# plot akrasi
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"],
    label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"],
    label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"],
    label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"],
    label="val_acc")
plt.title("Grafik Akurasi dan Loss")
plt.xlabel("Epochs #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

### Prediksi

In [ ]:
# Load Gambar
import matplotlib.image as mpimg
image_test = 'test/vinales.jpg'
image_show = mpimg.imread(image_test)
image = load_img(image_test, target_size=(96, 96))

In [ ]:
plt.imshow(image_show)

In [ ]:
# pra proses
image = np.array(image, dtype="float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)


In [ ]:
# klasifikasi input
print('[INFO] Mencari Kelas Gambar')
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

In [ ]:
# prediksi label + confidence score
label = "{}: {:.2f}%".format(label, proba[idx] * 100)
print(label)